# 🤖 AI-Powered FinOps Analysis with Gemini

This notebook demonstrates how to use the Gemini 2.5 Pro model to perform a deep, interactive analysis of your cloud cost and usage data.

## ⚠️ Important: Setup and Authentication

Before you begin, make sure you have completed the following steps:

1.  **Run the Setup Script**: From your terminal, in the project's root directory, run `bash scripts/setup_gcp_notebook.sh`.
2.  **Authenticate to Google Cloud**: Run `gcloud auth application-default login` in your terminal and follow the prompts.
3.  **Set Project ID**: Make sure your Google Cloud Project ID is set correctly in the `config.yaml` file or as an environment variable.
4.  **Enable Vertex AI API**: Ensure the Vertex AI API is enabled in your Google Cloud project.

## 📚 Step 1: Imports and Initialization

In [ ]:
import logging
import pandas as pd
from IPython.display import Markdown, display
from finops_analysis_platform.config_manager import ConfigManager
from finops_analysis_platform.data_loader import GCSDataLoader
from finops_analysis_platform.gemini_service import initialize_gemini, generate_with_code_execution

# Set up basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load configuration
config_manager = ConfigManager(config_path='config.yaml')
gcp_config = config_manager.get('gcp', {})
project_id = gcp_config.get('project_id')
location = gcp_config.get('location', 'us-central1')

# Initialize Gemini client
client = initialize_gemini(project_id=project_id, location=location)
model_id = 'gemini-2.0-flash-001'

print(f"✅ Gemini client initialized for project '{project_id}' in location '{location}'.")

## 📂 Step 2: Load Your Data

We'll load the billing and recommendations data from your GCS bucket.

In [ ]:
gcs_config = config_manager.get('gcs', {})
loader = GCSDataLoader(bucket_name=gcs_config.get('bucket_name'))
data = loader.load_all_data()
billing_df = data.get('billing')
recommendations_df = data.get('recommendations')

print("✅ Data loaded successfully.")

## 💬 Step 3: Interactive Analysis with Natural Language

Now, you can ask questions about your data in plain English. The Gemini model will generate and execute the code to answer your questions.

**Example Prompts:**
- "What are the top 10 most expensive services in the billing data?"
- "Show me a chart of the daily costs for the last 30 days."
- "What is the total potential monthly savings from the recommendations data?"

In [ ]:
prompt = "What are the top 5 most expensive services in the billing data?"

response = generate_with_code_execution(client, model_id, prompt)

# Display the results
for part in response.candidates[0].content.parts:
    if part.executable_code:
        display(Markdown(f"**Generated Code:**\n```python\n{part.executable_code.code}\n```"))
    if part.code_execution_result:
        display(Markdown(f"**Execution Result:**\n```\n{part.code_execution_result.output}\n```"))

## 💡 Step 4: Automated Insight Generation

We can also use Gemini to synthesize the findings from our analysis into a high-level summary.

In [ ]:
# To do this effectively, we would first run our CUD and recommendations analysis,
# then pass the results to Gemini.

# For this example, we'll use a prompt based on the insights we've already gathered.
insights_prompt = """
I have two sets of data: a CUD analysis and a list of cost-saving recommendations.

The CUD analysis shows significant potential savings from a mix of 1-year and 3-year CUDs.
The cost-saving recommendations show a total potential monthly savings of $49,860.53, with the largest savings coming from deleting unnecessary snapshots and unattached disks.

Please generate a short, actionable summary for a CFO, highlighting the key findings and recommending the next steps.
"""

summary_response = generate_with_code_execution(client, model_id, insights_prompt)

# Display the generated summary
for part in summary_response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))